# REDf (Recurrent Energy Demand forecasting) Model Implementation

This notebook implements the REDf model for energy demand forecasting using LSTM neural networks. The model processes hourly energy consumption data from multiple regions to predict future energy demand.

## Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
import os
import requests
import warnings

warnings.filterwarnings("ignore")

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

2025-04-17 22:08:06.106014: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 22:08:06.110039: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 22:08:06.118303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744907886.133020   77932 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744907886.137011   77932 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744907886.148767   77932 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

## Data Download and Preparation

This function downloads the required energy consumption datasets from GitHub if they don't already exist locally.

In [2]:

os.makedirs("data", exist_ok=True)
dataset_urls = {
    "AEP": "https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/refs/heads/master/data/AEP_hourly.csv",
    "COMED": "https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/refs/heads/master/data/COMED_hourly.csv",
    "DAYTON": "https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/refs/heads/master/data/DAYTON_hourly.csv",
    "PJME": "https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/refs/heads/master/data/PJME_hourly.csv",
}

dataset_paths = {}
for name, url in dataset_urls.items():
    file_path = f"data/{name}_hourly.csv"
    dataset_paths[name] = file_path
    if not os.path.exists(file_path):
        print(f"Downloading {name} dataset...")
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(file_path, "w") as f:
                f.write(response.text)
            print(f"{name} dataset downloaded successfully.")
        except Exception as e:
            print(f"Error downloading {name} dataset: {e}")
    else:
        print(f"{name} dataset already exists.")

AEP dataset already exists.
COMED dataset already exists.
DAYTON dataset already exists.
PJME dataset already exists.



## Algorithm 1: Data Pre-processing

Transform raw data into pre-processed data suitable for model training and evaluation.

### Steps:
1. **Input**: Raw Data `Draw`
2. **Output**: Pre-processed Data `Dpre`
3. **Procedure**:
    - **Step 4**: Load the raw data into the program: `Draw ← load_data()`
    - **Step 5**: Check for missing values and handle them accordingly: `Dpre ← handle_missing_values(Draw)`
    - **Step 6**: Check for outliers and handle them accordingly: `Dpre ← handle_outliers(Dpre)`
    - **Step 7**: Normalize the data: `Dpre ← normalize(Dpre)`
    - **Step 8**: Divide the data into training and testing sets: `(Dtrain, Dtest) ← split_data(Dpre)`
    - **Step 9**: Return the pre-processed data: `return Dpre`


In [ ]:
def preprocess_data(file_path):

    data = pd.read_csv(file_path)

    data["Datetime"] = pd.to_datetime(data["Datetime"])
    data.set_index("Datetime", inplace=True)
    
    energy_column = data.columns[0]
    energy_demand = data[energy_column].values.reshape(-1, 1)


    z_scores = (energy_demand - np.mean(energy_demand)) / np.std(energy_demand)
    abs_z_scores = np.abs(z_scores)
    outlier_indices = (abs_z_scores > 3).flatten()


    median_value = np.median(energy_demand)
    energy_demand[outlier_indices] = median_value

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(energy_demand)

    return scaled_data, scaler, energy_column

## Sequence Creation for Time Series

This section describes the function responsible for generating input-output sequences for time series forecasting. The model requires input sequences of several hours (24 in this case) to predict future energy demand accurately.

In [4]:
def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : (i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

## REDf Model Architecture

This section outlines the REDf model architecture as described in Algorithm 3 of the referenced paper.

### Algorithm 3: Proposed Forecasting Model

1. **Initialize the model**: `model = Sequential()`
2. **Add an LSTM layer with 200 units**: `model.add(LSTM(200, input_shape=(timesteps, features)))`
3. **Add a dropout layer with a rate of 0.1**: `model.add(Dropout(0.1))`
4. **Add another LSTM layer with 200 units**: `model.add(LSTM(200, return_sequences=False))`
5. **Add another dropout layer with a rate of 0.1**: `model.add(Dropout(0.1))`
6. **Add a fully connected layer with 1 unit**: `model.add(Dense(1))`
7. **Compile the model**: `model.compile(loss='mse', optimizer='adam')`
8. **Train the model on the training data**: `model.fit(X_train, y_train, epochs=10, batch_size=1000)`
9. **Generate predictions on the test data**: `y_pred = model.predict(X_test)`

The implementation uses the hyperparameters specified in the paper after grid search, including:
- LSTM units: 200
- Dropout rate: 0.1
- Epochs: 10
- Batch size: 1000

This approach skips grid search for hyperparameter tuning, as the final hyperparameters are directly adopted from the paper.

In [ ]:
def build_redf_model(input_shape):
    model = Sequential()
    model.add(LSTM(200, return_sequences=True, input_shape=(input_shape, 1)))
    model.add(Dropout(0.1))
    model.add(LSTM(200, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")
    return model


def train_redf(X_train, y_train, X_test, y_test, epochs=10, batch_size=1000):
    # Reshape input for LSTM [samples, time steps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    model = build_redf_model(X_train.shape[1])
    
    model.summary()
    
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
    )
    
    
    return model, history


## Training Workflow

This section outlines the workflow for training and evaluating the REDf model on multiple datasets. The process involves the following steps:

1. **Set Parameters**: Define key hyperparameters such as sequence length (`time_steps`), number of epochs, and batch size.
2. **Preprocess Data**: For each dataset, preprocess the raw data by handling missing values, removing outliers, and normalizing the energy demand values.
3. **Create Sequences**: Generate input-output sequences for time series forecasting using a sliding window approach.
4. **Split Data**: Divide the sequences into training (80%) and testing (20%) sets.
5. **Train the Model**: Train the REDf model using the training data and evaluate its performance on the testing data.
6. **Save the Model**: Save the trained model for each dataset for future use.

This workflow ensures a systematic approach to model training and evaluation, enabling reproducibility and consistent performance assessment across datasets.


In [ ]:

# Set parameters as specified in the paper
time_steps = 24  
epochs = 10     
batch_size = 1000  

for name, path in dataset_paths.items():
    print(f"\n{'='*50}")
    print(f"Processing {name} dataset...")
    print(f"{'='*50}")
    
    try:
        scaled_data, scaler, energy_column = preprocess_data(path)
        
        X, y = create_sequences(scaled_data, time_steps)
        
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        print(f"Training set shape: {X_train.shape}")
        print(f"Testing set shape: {X_test.shape}")
        
        model, history = train_redf(
            X_train, y_train, X_test, y_test, epochs=epochs, batch_size=batch_size
        )
        
        
        # Save the model
        model.save(f"{name}.h5")
        print(f"Model saved as {name}.h5")
        
        
    except Exception as e:
        print(f"Error processing {name} dataset: {e}")


## Model Loading and evaluation

1. **Load the Model**: The saved model can be loaded using the `load_model()` function from Keras.
2. **Inference**: The loaded model is used to make predictions on new or test data.
3. **Evaluation**: The predictions are evaluated using metrics such as MAE, RMSE, and R^2 to assess the model\'s performance.

In [ ]:
from tensorflow.keras.models import load_model
import glob

# Load all models from the ./models folder
model_files = glob.glob("./models/*.h5")
loaded_models = {}

for model_file in model_files:
    model_name = model_file.split("/")[-1].replace(".h5", "")
    loaded_models[model_name] = model_file

results = {}
for name, path in dataset_paths.items():
    print(f"\n{'='*50}")
    print(f"Processing {name} dataset...")
    print(f"{'='*50}")
    
    
    scaled_data, scaler, energy_column = preprocess_data(path)
    
    time_steps = 24
    X, y = create_sequences(scaled_data, time_steps)
    
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    model = load_model(loaded_models[name], custom_objects={"mse": tf.keras.losses.MeanSquaredError()})

    
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = (mae, rmse, r2)



Processing AEP dataset...


2025-04-17 22:08:20.803055: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


758/758 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step



Processing COMED dataset...
416/416 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step

Processing DAYTON dataset...


758/758 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step

Processing PJME dataset...


909/909 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step


## Final Output

This section summarizes the performance of the REDf model on each dataset.

In [ ]:
print("Dataset | MAE | RMSE | R²")
print("---------------------------------")
for name, metrics in results.items():
    mae, rmse, r2 = metrics
    print(f"{name.ljust(7)} | {mae:.4f} | {rmse:.4f} | {r2:.4f}")


Summary of Results:
------------------
Dataset | MAE | RMSE | R²
---------------------------------
AEP     | 0.0229 | 0.0337 | 0.9676
COMED   | 0.0347 | 0.0550 | 0.9118
DAYTON  | 0.0225 | 0.0358 | 0.9552
PJME    | 0.0241 | 0.0400 | 0.9450
